In [37]:
import pandas as pd

open_data_path = 'data/172_сalculate_pfound/open_task/'
hidden_data_path = 'data/172_сalculate_pfound/hidden_task/'

def get_pfound(data_path):
    qid_query = pd.read_csv(data_path + 'qid_query.tsv', sep="\t", names=['query_id', 'query_text'])
    qid_url_rating = pd.read_csv(data_path + 'qid_url_rating.tsv', sep="\t", names=['query_id', 'url', 'relevance'])
    hostid_url = pd.read_csv(data_path + 'hostid_url.tsv', sep="\t", names=['host_id', 'url'])
    
    qid_url_host = qid_url_rating.merge(hostid_url, on='url', how='left')
    qid_url_host = qid_url_host.sort_values(['query_id', 'host_id', 'relevance'], ascending=[True, True, False])
    qid_url_host = qid_url_host.drop_duplicates(subset=['query_id', 'host_id'], keep='first')
    qid_url_host = qid_url_host.sort_values(['query_id', 'relevance'], ascending=[True, False])
    
    pBreak = 0.15
    def calculate_pfound(group):
        top_documents = group.nlargest(10, 'relevance')
        p_found = 0
        p_look = 1
        for _, row in top_documents.iterrows():
            p_rel = row['relevance']
            p_found += p_look * p_rel
            p_look *= (1 - p_rel) * (1 - pBreak)
        return p_found
    
    pfound_scores = qid_url_host.groupby('query_id').apply(calculate_pfound).reset_index(name='pFound')
    max_pfound_query_id = pfound_scores.loc[pfound_scores['pFound'].idxmax(), 'query_id']
    max_pfound_query_text = qid_query.loc[qid_query['query_id'] == max_pfound_query_id, 'query_text'].values[0]
    
    return max_pfound_query_text

print('Open task query: {}'.format(get_pfound(open_data_path)))
print('Hidden task query: {}'.format(get_pfound(hidden_data_path)))

Open task query: гугл переводчик
Hidden task query: туту
